In [7]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import tqdm
import json

In [8]:
apiKey = "RGAPI-1ec3f4e3-9a9d-45e2-83a5-2c73600bb346"

In [9]:
with open("games_id.json","r") as f:
    games_id = json.load(f)
print(len(games_id))
print(games_id[0])

3607
EUN1_2339766487


In [10]:
def get_game_details_from_game_id(game_id, api):
    url = "https://europe.api.riotgames.com/tft/match/v1/matches/"+game_id+"?api_key="+api
    response = requests.get(url).json()['info']['participants']
    # print(np.array(response).shape)
    ans = []
    for i in response:
        player = []
        for champs in i['units']:
            # print(champs)
            # print(champs['name'], champs['items'], champs['tier'])
            player.append([champs['name'], champs['items'], champs['tier']])
        # print(player)
        ans.append(player)
    return ans

In [ ]:

games_id = list(games_id)
all_games = []
starttime = time.time()
for i in tqdm.tqdm(range(len(games_id)//20)):    
    starttime = time.time()
    for j in range(20):
        try:
            all_games.append(get_game_details_from_game_id(games_id[i*20+j], apiKey))
        except:
            pass
    time.sleep(25.0 - (time.time()-starttime))

print()
print(len(all_games))

with open('games.json', 'w') as f:
    json.dump(all_games, f)

  0%|                                                                                          | 0/180 [00:00<?, ?it/s]

# Scrapping full games from various divisions

In [11]:
def get_league(api, div, nr, page=1):
    '''
        returns: np.array of league summoners names
    '''
    url = "https://eun1.api.riotgames.com/tft/league/v1/entries/"+div+"/"+nr+"?page="+str(page)+"&api_key="+api
    
    response = requests.get(url)
    r = pd.DataFrame(response.json())
    return np.array(r).T[5]

plat1 = get_league(apiKey, "PLATINUM", "I", 1)

In [12]:
silver1 = get_league(apiKey, "SILVER", "I", 1)
iron1 = get_league(apiKey, "IRON", "I", 1)

In [13]:
def summoners_name_to_puuid(name, api):
    url = "https://eun1.api.riotgames.com/tft/summoner/v1/summoners/by-name/"+name+"?api_key="+api
    return  requests.get(url).json()['puuid']

In [14]:
def get_games_id_from_summoners_puuid(puuid, api, n=10):
    url = "https://europe.api.riotgames.com/tft/match/v1/matches/by-puuid/"+puuid+"/ids?count="+str(n)+"&api_key="+api
    return requests.get(url).json()

In [15]:
def get_game_details_from_game_id(game_id, api):
    url = "https://europe.api.riotgames.com/tft/match/v1/matches/"+game_id+"?api_key="+api
    response = requests.get(url).json()['info']['participants']
#     print(np.array(response))
    ans = []
    for i in response:
        player = []
        for champs in i['units']:
            # print(champs)
            # print(champs['name'], champs['items'], champs['tier'])
            player.append([champs['character_id'], champs['items'], champs['tier']])
        # print(player)
        synergies = []
        for trait in i['traits']:
            synergies.append([trait['name'], trait['num_units'], trait['tier_current'], trait['tier_total']])
        ans.append(player)
        ans.append(synergies)
        ans.append(i['total_damage_to_players'])
        ans.append(i['placement'])
    return ans

# get_game_details_from_game_id('EUN1_2378092721', apiKey)

In [9]:
plat1_players = get_league(apiKey, "PLATINUM", "I", 1)
np.array(plat1_players).shape
plat1_puuids = []
starttime = time.time()
for i in np.arange(0,50,10):
    starttime = time.time()
    for p in plat1_players[i:i+10]:
        plat1_puuids.append(summoners_name_to_puuid(p, apiKey))
    time.sleep(25.0 - (time.time()-starttime))

In [10]:
plat1_games_id = set()
for i in tqdm.tqdm(np.arange(0,50,10)):
    starttime = time.time()
    for p in plat1_puuids[i:i+10]:
        list_of_games = get_games_id_from_summoners_puuid(p, apiKey, 10)
        for l in list_of_games:
            plat1_games_id.add(l)
    time.sleep(25.0 - (time.time()-starttime))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:05<00:00, 25.00s/it]


In [11]:
# print(plat1_games_id)

In [12]:
plat1_games_id = list(plat1_games_id)
plat1_all_games = []
starttime = time.time()
for i in tqdm.tqdm(range(len(plat1_games_id)//20)):    
    starttime = time.time()
    for j in range(20):
        try:
            plat1_all_games.append(get_game_details_from_game_id(plat1_games_id[i*20+j], apiKey))
        except:
            pass
    time.sleep(25.0 - (time.time()-starttime))

print()
print(len(plat1_all_games))

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [10:25<00:00, 25.00s/it]


500


In [13]:
with open('data/plat1.json', 'w') as f:
    json.dump(list(plat1_all_games), f)

In [14]:
gold1_players = get_league(apiKey, "GOLD", "I", 1)
np.array(gold1_players).shape
gold1_puuids = []
starttime = time.time()
for i in np.arange(0,50,10):
    starttime = time.time()
    for p in gold1_players[i:i+10]:
        gold1_puuids.append(summoners_name_to_puuid(p, apiKey))
    time.sleep(25.0 - (time.time()-starttime))
    
gold1_games_id = set()
for i in tqdm.tqdm(np.arange(0,50,10)):
    starttime = time.time()
    for p in gold1_puuids[i:i+10]:
        list_of_games = get_games_id_from_summoners_puuid(p, apiKey, 10)
        for l in list_of_games:
            gold1_games_id.add(l)
    time.sleep(25.0 - (time.time()-starttime))
    

gold1_games_id = list(gold1_games_id)
gold1_all_games = []
starttime = time.time()
for i in tqdm.tqdm(range(len(gold1_games_id)//20)):    
    starttime = time.time()
    for j in range(20):
        try:
            gold1_all_games.append(get_game_details_from_game_id(gold1_games_id[i*20+j], apiKey))
        except:
            pass
    time.sleep(25.0 - (time.time()-starttime))

print()
print(len(gold1_all_games))

with open('data/gold1.json', 'w') as f:
    json.dump(list(gold1_all_games), f)

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [10:25<00:00, 25.00s/it]



500


In [15]:
gold1_players = get_league(apiKey, "SILVER", "I", 1)
np.array(gold1_players).shape
gold1_puuids = []
starttime = time.time()
for i in np.arange(0,50,10):
    starttime = time.time()
    for p in gold1_players[i:i+10]:
        gold1_puuids.append(summoners_name_to_puuid(p, apiKey))
    time.sleep(25.0 - (time.time()-starttime))
    
gold1_games_id = set()
for i in tqdm.tqdm(np.arange(0,50,10)):
    starttime = time.time()
    for p in gold1_puuids[i:i+10]:
        list_of_games = get_games_id_from_summoners_puuid(p, apiKey, 10)
        for l in list_of_games:
            gold1_games_id.add(l)
    time.sleep(25.0 - (time.time()-starttime))
    

gold1_games_id = list(gold1_games_id)
gold1_all_games = []
starttime = time.time()
for i in tqdm.tqdm(range(len(gold1_games_id)//20)):    
    starttime = time.time()
    for j in range(20):
        try:
            gold1_all_games.append(get_game_details_from_game_id(gold1_games_id[i*20+j], apiKey))
        except:
            pass
    time.sleep(25.0 - (time.time()-starttime))

print()
print(len(gold1_all_games))

with open('data/silver1.json', 'w') as f:
    json.dump(list(gold1_all_games), f)

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [10:25<00:00, 25.00s/it]



500


In [16]:
gold1_players = get_league(apiKey, "BRONZE", "I", 1)
np.array(gold1_players).shape
gold1_puuids = []
starttime = time.time()
for i in np.arange(0,50,10):
    starttime = time.time()
    for p in gold1_players[i:i+10]:
        gold1_puuids.append(summoners_name_to_puuid(p, apiKey))
    time.sleep(25.0 - (time.time()-starttime))
    
gold1_games_id = set()
for i in tqdm.tqdm(np.arange(0,50,10)):
    starttime = time.time()
    for p in gold1_puuids[i:i+10]:
        list_of_games = get_games_id_from_summoners_puuid(p, apiKey, 10)
        for l in list_of_games:
            gold1_games_id.add(l)
    time.sleep(25.0 - (time.time()-starttime))
    

gold1_games_id = list(gold1_games_id)
gold1_all_games = []
starttime = time.time()
for i in tqdm.tqdm(range(len(gold1_games_id)//20)):    
    starttime = time.time()
    for j in range(20):
        try:
            gold1_all_games.append(get_game_details_from_game_id(gold1_games_id[i*20+j], apiKey))
        except:
            pass
    time.sleep(25.0 - (time.time()-starttime))

print()
print(len(gold1_all_games))

with open('data/bronze1.json', 'w') as f:
    json.dump(list(gold1_all_games), f)

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [10:00<00:00, 25.00s/it]



480


In [17]:
gold1_players = get_league(apiKey, "IRON", "I", 1)
np.array(gold1_players).shape
gold1_puuids = []
starttime = time.time()
for i in np.arange(0,50,10):
    starttime = time.time()
    for p in gold1_players[i:i+10]:
        gold1_puuids.append(summoners_name_to_puuid(p, apiKey))
    time.sleep(25.0 - (time.time()-starttime))
    
gold1_games_id = set()
for i in tqdm.tqdm(np.arange(0,50,10)):
    starttime = time.time()
    for p in gold1_puuids[i:i+10]:
        list_of_games = get_games_id_from_summoners_puuid(p, apiKey, 10)
        for l in list_of_games:
            gold1_games_id.add(l)
    time.sleep(25.0 - (time.time()-starttime))
    

gold1_games_id = list(gold1_games_id)
gold1_all_games = []
starttime = time.time()
for i in tqdm.tqdm(range(len(gold1_games_id)//20)):    
    starttime = time.time()
    for j in range(20):
        try:
            gold1_all_games.append(get_game_details_from_game_id(gold1_games_id[i*20+j], apiKey))
        except:
            pass
    time.sleep(25.0 - (time.time()-starttime))

print()
print(len(gold1_all_games))

with open('data/iron1.json', 'w') as f:
    json.dump(list(gold1_all_games), f)

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [09:35<00:00, 25.00s/it]



444


In [16]:
gold1_players = get_league(apiKey, "DIAMOND", "I", 1)
np.array(gold1_players).shape
gold1_puuids = []
starttime = time.time()
for i in np.arange(0,50,10):
    starttime = time.time()
    for p in gold1_players[i:i+10]:
        gold1_puuids.append(summoners_name_to_puuid(p, apiKey))
    time.sleep(25.0 - (time.time()-starttime))
    
gold1_games_id = set()
for i in tqdm.tqdm(np.arange(0,50,10)):
    starttime = time.time()
    for p in gold1_puuids[i:i+10]:
        list_of_games = get_games_id_from_summoners_puuid(p, apiKey, 10)
        for l in list_of_games:
            gold1_games_id.add(l)
    time.sleep(25.0 - (time.time()-starttime))
    

gold1_games_id = list(gold1_games_id)
gold1_all_games = []
starttime = time.time()
for i in tqdm.tqdm(range(len(gold1_games_id)//20)):    
    starttime = time.time()
    for j in range(20):
        try:
            gold1_all_games.append(get_game_details_from_game_id(gold1_games_id[i*20+j], apiKey))
        except:
            pass
    time.sleep(25.0 - (time.time()-starttime))

print()
print(len(gold1_all_games))

with open('data/diamond1.json', 'w') as f:
    json.dump(list(gold1_all_games), f)

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [10:00<00:00, 25.00s/it]



480
